In [1]:
import sys
sys.path.append('./rtaUtils')

from rtaUtils import data_loading, data_preparation, paths

In [2]:
numeric_feat = [
    'latitude', 'longitude', 'altitude', 'delay', 'vspeed', 'speed', 
    'day_of_week', 'track', 'wind_dir_degrees', 'wind_speed_kt', 
    'visibility_statute_mi', 'max_temp', 'min_temp', 'hav_distance',
    'cloud_base_ft_agl'
]
categoric_feat = [
    'time_of_day', 'operator', 'aerodromeOfDeparture', 'aircraftType', 'cloud_type', 'sky_cover'
]
objective = ['RTA']
num_features   = len(numeric_feat+categoric_feat)

ts_features = ['latitude', 'longitude', 'altitude', 'vspeed', 'speed', 'track', 'hav_distance']
nts_features = ['departureDelay', 'day_of_week', 'wind_dir_degrees','wind_speed_kt', 
                'visibility_statute_mi', 'max_temp', 'min_temp', 'time_of_day', 'operator', 
                'aerodromeOfDeparture', 'sky_status', 'clouds']

feat_dict = {
    'numeric':numeric_feat,
    'categoric':categoric_feat,
    'objective':objective,
}

In [5]:
data_loading.load_final_data('202201', 'train', '*', 30).iloc[0]

fpId                                  AT05414795
icao24                                    40697C
callsign                                  BAW456
latitude                               51.477402
longitude                                -0.4745
speed                                      141.0
vspeed                                    2689.0
ground                                     False
track                                        267
actualTakeOffTime                     1641015840
delay                                          0
aerodromeOfDeparture                        EGLL
aerodromeOfDestination                      LEMD
operator                                     BAW
aircraftType                                A320
flightDate                            2022-01-01
touchdown                           1641022933.0
RTA                                         7048
vectorId                  20220101-40697C-000014
reportId                      LEMD-1641009600-01
timestamp           

# Downsample

In [ ]:
sampling = 15

months = [f'2022{str(x).rjust(2,"0")}' for x in range(1,10)]
airport = '*'

for month in months:
    train_data = data_loading.load_final_data(month, 'train', airport)
    test_data  = data_loading.load_final_data(month, 'test', airport)
    val_data   = data_loading.load_final_data(month, 'val', airport)
        
    start_train = train_data.shape[0]
    start_test = test_data.shape[0]
    start_val = val_data.shape[0]
    
    train_data = data_preparation.sample_data(train_data, sampling)
    test_data = data_preparation.sample_data(test_data, sampling)
    val_data = data_preparation.sample_data(val_data, sampling)
    
    print(f'{month} | ' + 
          f'train: {start_train:>9,} -> {train_data.shape[0]:>9,} | ' + 
          f'test: {start_test:>7,} -> {test_data.shape[0]:>7,} | ' + 
          f'val: {start_val:>6,} -> {val_data.shape[0]:>6,}')
    
    train_data.to_parquet(paths.sampled_data_path / f's{sampling}/{month}.train.parquet')
    test_data.to_parquet(paths.sampled_data_path / f's{sampling}/{month}.test.parquet')
    val_data.to_parquet(paths.sampled_data_path / f's{sampling}/{month}.val.parquet')

# Windowing

In [ ]:
sampling = 60
lookback = 32

months = [f'2022{str(x).rjust(2,"0")}' for x in range(1,10)]
airport = '*'

for month in months:
    data_preparation.generate_save_windows(month, lookback, sampling, feat_dict, airport)